In [ ]:
import pandas as pd
df= pd.read_csv('urls_scan_1.csv')
df.head()

,text,team,about,product,service
0,office moving services,0,0,0,1
1,our services,0,0,0,1
2,services,0,0,0,1
3,about us,0,1,0,0
4,packing services,0,0,0,1


In [ ]:
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)
print(df.head())

                     text  team  about  product  service
0  office moving services     0      0        0        1
1            our services     0      0        0        1
2                services     0      0        0        1
3                about us     0      1        0        0
4        packing services     0      0        0        1


In [ ]:
df.to_csv('urls_scan_1.csv', index=False)
df.head()

,text,team,about,product,service
0,office moving services,0,0,0,1
1,our services,0,0,0,1
2,services,0,0,0,1
3,about us,0,1,0,0
4,packing services,0,0,0,1


In [ ]:
pip install transformers datasets scikit-learn torch beautifulsoup4 requests


In [11]:
import pandas as pd
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
import joblib

# Load dataset
df = pd.read_csv('urls_scan_1.csv')
df_small = df.sample(n=20000, random_state=42)

# Extract path from 'text' column (assumed to be full URLs)
def extract_url_path(url):
    try:
        parsed = urlparse(url)
        path = parsed.path or '/'
        if parsed.fragment:
            frag = parsed.fragment
            if frag.startswith('!/'):
                path = '/' + frag[2:]
            elif frag.startswith('/'):
                path = frag
            elif not parsed.path or parsed.path == '/':
                path = '/' + frag
        return path
    except:
        return ''

df_small['text'] = df_small['text'].apply(extract_url_path)
df_small = df_small[df_small['text'].str.strip() != '']

label_cols = ['team', 'about', 'product', 'service']
df_small[label_cols] = df_small[label_cols].fillna(0).astype(int)

# 🔥 Manually add key training examples to improve prediction
extra_data = pd.DataFrame({
    'text': ['/aboutus', '/about', '/team', '/contactus'],
    'team':    [0, 0, 1, 0],
    'about':   [1, 1, 0, 1],
    'product': [0, 0, 0, 0],
    'service': [0, 0, 0, 1]
})

df_small = pd.concat([df_small, extra_data], ignore_index=True)

# Custom tokenizer: split path into meaningful tokens
def custom_tokenizer(path):
    return path.strip('/').replace('-', '/').replace('_', '/').split('/')

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000, tokenizer=custom_tokenizer)
X = df_small['text'].tolist()
y = df_small[label_cols].values
X_tfidf = vectorizer.fit_transform(X)

# Split and train
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.1, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf = OneVsRestClassifier(rf)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred, target_names=label_cols))

# Save model and vectorizer
joblib.dump(clf, 'random_forest_web_classifier.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("✅ Model and vectorizer saved successfully.")


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

        team       0.96      0.44      0.61        54
       about       0.98      0.67      0.80       290
     product       0.61      0.96      0.75       686
     service       0.98      0.25      0.40       503

   micro avg       0.70      0.65      0.68      1533
   macro avg       0.88      0.58      0.64      1533
weighted avg       0.81      0.65      0.64      1533
 samples avg       0.66      0.66      0.66      1533

✅ Model and vectorizer saved successfully.


In [13]:
import joblib
from urllib.parse import urlparse
import ipywidgets as widgets
from IPython.display import display, clear_output

clf = joblib.load('random_forest_web_classifier.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
label_cols = ['team', 'about', 'product', 'service']

input_box = widgets.Text(
    value='',
    placeholder='Enter a URL (e.g., https://example.com/about)',
    description='URL:',
    layout=widgets.Layout(width='100%')
)

output = widgets.Output()

def extract_url_path(url):
    try:
        parsed = urlparse(url)
        path = parsed.path or '/'
        if parsed.fragment:
            frag = parsed.fragment
            if frag.startswith('!/'):
                path = '/' + frag[2:]
            elif frag.startswith('/'):
                path = frag
            elif not parsed.path or parsed.path == '/':
                path = '/' + frag
        return path
    except:
        return ''


def on_enter(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()
            url = change['new'].strip()
            if not url:
                print("Please enter a URL.")
                return

            path = extract_url_path(url)
            if not path:
                print("Could not extract a valid path from the URL.")
                return

            X_input = vectorizer.transform([path])
            prediction = clf.predict(X_input)[0]
            predicted_labels = [label_cols[i] for i, val in enumerate(prediction) if val == 1]

            print(f"\nURL: {url}")
            print(f"Extracted Path: {path}")
            if predicted_labels:
                print(f"\nPredicted Labels: {', '.join(predicted_labels)}")
            else:
                print("\nNo matching labels predicted.")

input_box.observe(on_enter)
display(input_box, output)


Text(value='', description='URL:', layout=Layout(width='100%'), placeholder='Enter a URL (e.g., https://exampl…

Output()